In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Saimese Training
**Adapt from**

https://www.kaggle.com/code/tanulsingh077/siamese-style-training-efficient-net-b0/notebook

https://datahacker.rs/019-siamese-network-in-pytorch-with-application-to-face-similarity/




In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision

import sys
sys.path.append('../input/pytorch-image-models')
import timm

from tqdm import tqdm
# timm.list_models(pretrained=True)

In [ ]:
# Constant
TRAIN_PATH="../input/shopee-product-matching/train.csv"
TRAIN_SIAMESE_PATH="../input/code-for-data-generation-for-siamese-training/siamese_data.csv"
TEST_PATH="../input/shopee-product-matching/test.csv"
TRAIN_IMAGE_PATH="../input/shopee-product-matching/train_images/" # + image id
TEST_IMAGE_PATH="../input/shopee-product-matching/test_images/"

BASE_MODEL = "resnet18"

DIM = (512, 512)
N_CLASS = 11014

LR = 5e-4
BATCH_SIZE = 16
EPOCHS = 1

In [ ]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
        torch.backends.cudnn.benchmark = True
    else:
        device = torch.device('cpu') # don't have GPU 
    return device
DEVICE = get_device()
print(DEVICE)

In [ ]:
def preprocess():
    train_df = pd.read_csv(TRAIN_PATH)
    train_siamese_df = pd.read_csv(TRAIN_SIAMESE_PATH)
    test_df = pd.read_csv(TEST_PATH)
    
    return train_df, train_siamese_df, test_df

train_df, train_siamese_df, test_df = preprocess()
N_CLASS = train_df['label_group'].nunique()
print("n_trian: {} n_unique: {} n_per_images: {}"
         .format(train_df['label_group'].shape[0],
                 train_df['label_group'].nunique(),
                 train_df['label_group'].shape[0]/train_df['label_group'].nunique())
         )
# print(train_siamese_df.head())
sample_img = cv2.imread(TRAIN_IMAGE_PATH+train_df.loc[0, 'image'])
sample_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)
plt.imshow(sample_img)
plt.show()

In [ ]:
'''Transform only image'''
def train_transforms():
    return A.Compose([
        A.Normalize(
            max_pixel_value=255.0, always_apply=True
        ),
        ToTensorV2()
    ])

def validate_transforms():
    return A.Compose([
        A.Normalize(
            max_pixel_value=255.0, always_apply=True
        ),
        ToTensorV2()
    ])

x = train_transforms()(image=sample_img)['image']
print(x, x.shape, x.min(), x.max())

In [ ]:
class ShopeeDataset(Dataset):
    def __init__(self, image_0,image_1,labels,dim=DIM, augmentation=None, is_train=True):
        self.image_0 = image_0
        self.image_1 = image_1
        self.labels = labels
        self.dim = dim
        self.is_train = is_train
        self.augmentation = augmentation
        
    def get_image(self, image_path, is_train=True):
        if self.is_train:
            img = cv2.imread(os.path.join(TRAIN_IMAGE_PATH, image_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.dim)
        else:
            img = cv2.imread(os.path.join(TEST_IMAGE_PATH, image_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.dim)
        return img
        
    def __len__(self):
        return len(self.image_0)
    
    def __getitem__(self, idx):
        img_0 = self.image_0[idx]
        img_1 = self.image_1[idx]
        
        
        img_0 = self.get_image(img_0)
        img_1 = self.get_image(img_1)
        
        
        if self.augmentation:
            tmp_0 = self.augmentation(image=img_0)
            tmp_1 = self.augmentation(image=img_1)
            
            img_0 = tmp_0['image']
            img_1 = tmp_1['image']
            
        
        if self.is_train:
            label = torch.tensor(self.labels[idx], dtype=torch.float32)
        else:
            label = self.labels[idx]
        
        return img_0, img_1, label

        

In [ ]:
class ImageModel(nn.Module):
    def __init__(self, model_name=BASE_MODEL, pretrained=True):
        super().__init__()
        
        if model_name == "efficientnet_b0":
            self.backbone_model = timm.create_model(model_name, pretrained=pretrained)
            n_features = self.backbone_model.classifier.in_features

            self.backbone_model.global_pool = nn.Identity()
            self.backbone_model.classifier = nn.Identity()

            self.pooling = nn.AdaptiveAvgPool2d(1)
        
        if model_name=="resnet18":
#             self.backbone_model = timm.create_model(model_name, pretrained=pretrained)
            self.backbone_model = torchvision.models.resnet18(pretrained=True)
            n_features =  self.backbone_model.fc.in_features
            self.backbone_model.fc = nn.Linear(n_features, 512)
            print("resnet18 in_features: {}".format(n_features))
            
        
        self.classifier = nn.Sequential(
            nn.Linear(n_features, 256),
            nn.ReLU(),
            nn.Linear(256, 2) # 2 is for same or not
        )
        
    def forward_once(self, x):
        tmp = x.size(0)
        output = self.backbone_model(x)
        if self.backbone_model == "efficientnet_b0":
            output = self.pooling(output).view(tmp, -1)
        if self.backbone_model == "resnet18":
            output = nn.Flatten(output)
        output = self.classifier(output)
        return output
        
        
        
    '''Training siamese model'''
    def forward(self, image_0, image_1):
        output_0 = self.forward_once(image_0)
        output_1 = self.forward_once(image_1)
        return output_0, output_1

i_model = ImageModel()
print(i_model)
if torch.cuda.is_available():
    i_model = i_model.to(DEVICE)

In [ ]:
@torch.no_grad()
def get_dissimilarity(model, anchor_img, img):
    output_0, output_1 = model(anchor_img, img)
    dist = F.pairwise_distance(output_0, output_1)
    return dist

In [ ]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)

        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                    (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [ ]:
def train(model, train_dl, optimizer, criterion,scheduler=None,epochs=EPOCHS):
    if torch.cuda.is_available():
        model = model.to(DEVICE)
        criterion = criterion.to(DEVICE)
        print("Using Cuda", next(model.parameters()).device)
    
    for epoch in range(epochs):
        for img_0, img_1, label in tqdm(train_dl):
            
            optimizer.zero_grad()
            
            if torch.cuda.is_available():
                img_0 = img_0.to(DEVICE)
                img_1 = img_1.to(DEVICE)
                label = label.to(DEVICE)
                
            output_0, output_1, = model(img_0, img_1)
            
            loss = criterion(output_0, output_1, label)
            loss.backward()
            optimizer.step()
        
        if scheduler is not None:
            scheduler.step()
            
        with torch.no_grad():
            print(loss)

In [ ]:
train_siamese_ds = ShopeeDataset(
    image_0 = train_siamese_df['image_1'].values.tolist(),
    image_1 = train_siamese_df['image_2'].values.tolist(),
    labels = train_siamese_df['label'].values.tolist(),
    dim = DIM,
    augmentation=train_transforms(),
)

train_siamese_dl = DataLoader(
    train_siamese_ds,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers=2
)

optimizer = optim.Adam(i_model.parameters(), lr=LR)

criterion = ContrastiveLoss()

scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=5e-3, epochs=EPOCHS, steps_per_epoch=len(train_siamese_dl))

# for img0, img1, label in train_siamese_dl:
#     print(img0[0].shape, img1[0].shape, label)
#     plt.imshow(img0[0])
#     plt.imshow(img1[0])
#     plt.show()
#     break
train(i_model, train_siamese_dl, optimizer, criterion, None)

In [ ]:
torch.save(i_model.state_dict(), "model_1.bin")

In [ ]:
def preview_predict(model, n_of_sample_batch=1):
    if torch.cuda.is_available():
        model = model.to(DEVICE)
    cnt = 0
    for img_0, img_1, label in train_siamese_dl:
        if torch.cuda.is_available():
                img_0 = img_0.to(DEVICE)
                img_1 = img_1.to(DEVICE)
                label = label.to(DEVICE)
        cnt += 1
        for i in range(BATCH_SIZE):
            x = img_0[i].view(-1, 3, 512, 512)
            y = img_1[i].view(-1, 3, 512, 512)
            print("predict: {}\t lable: {}".format(get_dissimilarity(model, x, y), label[i]))
        if cnt>=n_of_sample_batch: break
              
model_x = ImageModel()
model_x.load_state_dict(torch.load("model_1.bin"))
preview_predict(i_model)

In [ ]:
@torch.no_grad()
def check(image_model):
    for img0, img1, label in train_siamese_dl:
        img0, img1 = img0[0].permute(1, 2, 0).numpy(), torch.permute(img1[0], [1, 2, 0]).cpu().detach().numpy()
        
        img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
        plt.imshow(img0)
        plt.show()
#         print(image_model(img0, img1))
        break
check(i_model)

In [ ]:
model_x = ImageModel()
model_x.load_state_dict(torch.load("model_1.bin"))

@torch.no_grad()
def something(model):
    result_df = pd.DataFrame(columns=['posting_id', 'ans'])
    result_df['posting_id'] = test_df['posting_id']
    test_ds = ShopeeDataset(
        image_0 = test_df['image'].values.tolist(),
        image_1 = test_df['image'].values.tolist(),
        labels = test_df['posting_id'].values.tolist(),
        dim = DIM,
        augmentation=train_transforms(),
        is_train=False
    )

    test_dl = DataLoader(
        test_ds,
        shuffle=True,
        num_workers=2
    )

    tmpp = [[], [], []]
    for idx, anchor in tqdm(enumerate(test_ds)):
        print(anchor[1].shape)
        img =  torch.permute(anchor[1], [1, 2, 0]).cpu().detach().numpy()
        plt.imshow(img)
        plt.show()
#         for img, _, _ in test_ds:
#             print(model(anchor, img))
    #         list[idx].append(model_i(anchor, img))
    return tmpp

print(something(model_x))